In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from autokeras import StructuredDataClassifier
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")


path = "data/"
#df_train = pd.read_csv(path+"train.csv")
df_train = pd.read_csv(path+"train_preprocess_binarized_hotEncoded_full.csv")

df_test = pd.read_csv(path+"test.csv")



In [2]:
del df_train['Unnamed: 0']	

In [3]:
y = df_train["Survived"]
del df_train["Survived"]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size = 0.10, random_state = 42)


In [5]:
X_train.head(2)

,Sex,Age,Fare,Embarked_C,Embarked_Q,Embarked_S,SibSp_0,SibSp_1,SibSp_2,SibSp_3,...,Pclass_2,Pclass_3,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Parch_9
51994,0,3.0,65.64,1,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
77540,1,5.0,25.95,0,0,1,0,1,0,0,...,0,1,0,1,0,0,0,0,0,0


In [6]:
X_train.shape

(90000, 24)

In [7]:
X_test.head(2)

,Sex,Age,Fare,Embarked_C,Embarked_Q,Embarked_S,SibSp_0,SibSp_1,SibSp_2,SibSp_3,...,Pclass_2,Pclass_3,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Parch_9
75721,0,57.0,8.32,1,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,0,0
80184,1,26.0,6.17,0,0,1,1,0,0,0,...,0,1,1,0,0,0,0,0,0,0


In [8]:
X_test.shape

(10000, 24)

In [12]:
# define the search
search = StructuredDataClassifier(max_trials=50, tuner='random', objective='val_accuracy', overwrite=True, project_name='StructuredCL_fullEncoding',
                                  metrics=[tf.keras.metrics.CategoricalAccuracy(),'accuracy',
                                tf.keras.metrics.Precision(),
                                tf.keras.metrics.Recall(),
                                tf.keras.metrics.PrecisionAtRecall(0.5)])

In [13]:

#csv_logger = CSVLogger('training.log', separator=";", append=True))

my_callbacks = [
    #tf.keras.callbacks.EarlyStopping(patience=2),
    #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.CSVLogger('models/autoKears_CSVlogging.log', separator=";", append=True)
]
#model.fit(X_train, Y_train, callbacks=my_callbacks)

In [ ]:
# perform the search
search.fit(x=X_train, y=y_train, verbose=1)#, callbacks=my_callbacks)

Trial 13 Complete [00h 01m 11s]
val_accuracy: 0.6965160369873047

Best val_accuracy So Far: 0.7719835042953491
Total elapsed time: 00h 54m 42s

Search: Running Trial #14

Hyperparameter    |Value             |Best Value So Far 
structured_data...|True              |False             
structured_data...|True              |False             
structured_data...|3                 |1                 
structured_data...|512               |1024              
structured_data...|0.25              |0                 
structured_data...|128               |256               
classification_...|0.5               |0                 
optimizer         |adam_weight_decay |adam              
learning_rate     |0.1               |0.001             
structured_data...|256               |64                



In [15]:
# evaluate the model
loss, acc = search.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.3f' % acc)

ValueError: too many values to unpack (expected 2)

In [16]:
# get the best performing model
model = search.export_model()

In [17]:
# summarize the loaded model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 25)                0         
_________________________________________________________________
normalization (Normalization (None, 25)                51        
_________________________________________________________________
dense (Dense)                (None, 512)               13312     
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
re_lu (ReLU)                 (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0     

In [18]:
# save the best performing model to file
model.save('models/autokeras_model/fullEncoding/model_v1', overwrite=True, include_optimizer=True)

INFO:tensorflow:Assets written to: autokeras_model/fullEncoding/model_v1\assets


In [32]:
import tensorflow as tf
import keras
import numpy as np

In [33]:
loaded = keras.models.load_model("models/autokeras_model/fullEncoding/model_v1")

In [34]:
loaded.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 25)                0         
_________________________________________________________________
normalization (Normalization (None, 25)                51        
_________________________________________________________________
dense (Dense)                (None, 512)               13312     
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
re_lu (ReLU)                 (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0     